In [27]:
%run /home/analysis_user/New_trap_code/Tools/basic_packages.py

In [28]:
%matplotlib widget

### Discharge data

In [29]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 2
decimate = 10
drive_freq = 71
fsamp = 5000
def func1_scale(i):
    bb = step_cal_files[i]
        
    freq2 = np.fft.rfftfreq(len(bb.electrode_data[0]), d=1./5000)
    fft_angles = np.angle(np.fft.rfft(bb.electrode_data[0]))
    phi_tmp2 = (fft_angles[freq2==drive_freq]+np.pi/2)%(2*np.pi)
    print(phi_tmp2,np.std(bb.electrode_data[0])*np.sqrt(2))

    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp2, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [-1, 1], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': False}

    frequency = fit_kwargs['f']

    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.x2)[::decimate]
    
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    return m1_tmp.values[0],m1_tmp.values[2]

In [30]:
step_cal_dir = r'/data/new_trap/20210819/Bead1/Discharge/FineDischarge/'
step_cal_files = load_dir(step_cal_dir, start_file=0)

100%|██████████| 110/110 [00:02<00:00, 42.96it/s]

110  files in folder
110  files loaded


In [31]:
amps3 = []
for i in range(len(step_cal_files)):
    print(i,'/',len(step_cal_files))
    amps3.append(func1_scale(i)[0])

0 / 110
[0.08159127] 9.996258641443987
1 / 110
[0.08158181] 9.996261562905714
2 / 110
[0.08158551] 9.996262826363902
3 / 110
[0.0815914] 9.996261024164689
4 / 110
[0.08160914] 9.996255982376793
5 / 110
[0.08158232] 9.996291576691956
6 / 110
[0.08159683] 9.99629738460346
7 / 110
[0.08161135] 9.996294209095058
8 / 110
[0.08161813] 9.996288186989338
9 / 110
[0.08161548] 9.996293299986304
10 / 110
[0.08160144] 9.996121181390475
11 / 110
[0.08156281] 9.996121232412026
12 / 110
[0.08157338] 9.99611902714073
13 / 110
[0.08157973] 9.996123751130634
14 / 110
[0.08158879] 9.996118867766942
15 / 110
[0.08159015] 9.996293898343522
16 / 110
[0.08159399] 9.996295801114082
17 / 110
[0.08157734] 9.996296844269922
18 / 110
[0.08160145] 9.996291026971965
19 / 110
[0.08159037] 9.996292761938781
20 / 110
[0.08161575] 9.996292540091673
21 / 110
[0.08159737] 9.996293891203111
22 / 110
[0.08160346] 9.996294403395577
23 / 110
[0.08160373] 9.996296160912763
24 / 110
[0.0816174] 9.99629768840838
25 / 110
[0.081

In [38]:
scale = np.abs(np.mean(amps3[70:80]))/8
print("Scale is ", scale)
bb = step_cal_files[-1]
voltage = np.std(bb.electrode_data[0])*np.sqrt(2)
print("voltage: ", voltage)

Scale is  2.339906716277429e-05
voltage:  9.996262322298485


In [39]:
# amps = np.array(amps)
amps3 = np.array(amps3)
_,ax = plt.subplots(figsize=(9.4,4))

# Perrinf
ax.scatter(range(len(amps3)), amps3/scale, label='fit phase')
# ax.scatter(range(len(amps3)), amps3/scale, label='fit fixed-phases')

ax.set(xlabel='dataset', ylabel='Electron charges', ylim=(-22, 0.1), xlim=(30,None))
ax.grid(axis='y')
ax.minorticks_on()
ax.grid(which='major', linestyle='-', linewidth='0.5', color='red')
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
# ax.legend()
from matplotlib.ticker import MaxNLocator
ax.yaxis.set_major_locator(MaxNLocator(integer=True))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [40]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 2
decimate = 10
drive_freq = 71
phase_calib = 0
# phase_calib = -0.0442

def get_amp(i):
    
    fname = r'/data/new_trap/20210819/Bead1/Neutrality/200V_X_SPINYZ_58p5kHz_100V_20210820/Discharge_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)


    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phase_calib, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [-1, 1], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': False}
    
    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.x2)[::decimate]
    
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    return m1_tmp.values[0]

In [41]:
i=0
get_amp(i)/scale

-0.04717982215115141

In [42]:
i = 0
fname = r'/data/new_trap/20210819/Bead1/Neutrality/200V_X_SPINYZ_58p5kHz_100V_20210820/Discharge_'+str(i)+'.h5'
bb = BeadDataFile.BeadDataFile(fname=fname)
voltage0_200 = np.std(bb.electrode_data[0])*np.sqrt(2)*200
voltage1_200 = np.std(bb.electrode_data[1])*np.sqrt(2)*200
bias2 = np.mean(bb.electrode_data[0])*200
print("voltages, bias: ", voltage0_200, voltage1_200, bias2)

voltages, bias:  200.04036193179343 0.04137692997338144 0.42040024668424963


In [43]:
voltage, voltage0_200, scale

(9.996262322298485, 200.04036193179343, 2.339906716277429e-05)

In [45]:
amps_data_200V = np.array(Parallel(n_jobs=20)(delayed(get_amp)(i) for i in tqdm(range(1800))))

100%|██████████| 1800/1800 [00:07<00:00, 254.88it/s]


In [46]:
data0 = amps_data_200V/scale*voltage/voltage0_200

dataPlus=data0[::4]
dataMinus=data0[2::4]

_, ax= plt.subplots()
ax.scatter(np.arange(len(dataPlus)), dataPlus, label='X-axis, plus, 200V, YZ-spin, with-height-fb')
ax.scatter(np.arange(len(dataMinus)), dataMinus, label='X-axis, minus, 200V, YZ-spin, with-height-fb')
ax.plot(np.arange(0,1000), np.arange(0,1000)*0, '--r')
ax.set(xlabel='Dataset', ylabel=r'$\epsilon$')
ax.legend()
print('mean: ', np.mean(dataPlus), 'std: ', np.std(dataPlus)/np.sqrt(len(dataPlus)))
print('mean: ', np.mean(dataMinus), 'std: ', np.std(dataMinus)/np.sqrt(len(dataMinus)))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

mean:  -0.0015166563609849495 std:  0.0002798156543739575
mean:  0.006923128961176383 std:  0.0012722564951503997


In [19]:
-0.007713032166712999/0.03923405818351312 

-0.19659022094110468

In [24]:
-0.0016038530141878442/0.007201926298527419

-0.22269778219138187